# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.


In [143]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'
# CSV path
csv_file_path = 'SDW2023.csv'

## **E**xtract

Extraindo informações INTRADAY utilizando a api alphavantage para analise do exemplo #IBM.

In [144]:
import requests
import json
import pandas as pd
import numpy as np

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# documents about this api https://www.alphavantage.co/documentation/

# Input the company that you want. Ex.:IBM.
symbol = input("")

# You can load as csv datatype as you prefer, more info in documents link above.
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=60min&outputsize=full&apikey=BXINLY1AXIHM6ALZ'

r = requests.get(url)

data = r.json()

print(data)

ibm
{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume', '2. Symbol': 'ibm', '3. Last Refreshed': '2023-08-18 19:00:00', '4. Interval': '60min', '5. Output Size': 'Full size', '6. Time Zone': 'US/Eastern'}, 'Time Series (60min)': {'2023-08-18 19:00:00': {'1. open': '141.4100', '2. high': '141.5000', '3. low': '141.2800', '4. close': '141.5000', '5. volume': '980290'}, '2023-08-18 18:00:00': {'1. open': '141.2700', '2. high': '141.5000', '3. low': '141.2700', '4. close': '141.5000', '5. volume': '980792'}, '2023-08-18 17:00:00': {'1. open': '141.2700', '2. high': '141.5000', '3. low': '141.2700', '4. close': '141.5000', '5. volume': '2174'}, '2023-08-18 16:00:00': {'1. open': '141.4500', '2. high': '141.4900', '3. low': '140.9200', '4. close': '141.2600', '5. volume': '3213967'}, '2023-08-18 15:00:00': {'1. open': '141.1600', '2. high': '141.8300', '3. low': '141.0600', '4. close': '141.4600', '5. volume': '823165'}, '2023-08-18 14:00:00': {'1. o

In [145]:
# Read user IDs from the CSV file
df = pd.read_csv(csv_file_path)
user_ids = df['UserID'].tolist()

def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve user data for ID {id}")
        return None

# Fetch user data for each user ID and store in the users list
users = []
for id in user_ids:
    user_data = get_user(id)
    if user_data:
        users.append(user_data)

# Print the list of users
print(json.dumps(users, indent=2))

# Extract data to a list
time_series_data = list(data['Time Series (60min)'].values())

Failed to retrieve user data for ID 2
Failed to retrieve user data for ID 3
Failed to retrieve user data for ID 4
[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        "id": 5,
        "icon": "https://digitalinnovati

## **T**ransform

Mean between low and high with 60min gap, calculate the deviation for the period, append at Swagger api and save as csv for further analysis.

In [146]:
# Fills in None values ​​for missing 'low' and 'high'
for item in time_series_data:
    if '3. low' not in item:
        item['3. low'] = None
    if '2. high' not in item:
        item['2. high'] = None

# Converts the values ​​of 'low' and 'high' to float and creates a DataFrame
df = pd.DataFrame(time_series_data)
df['3. low'] = df['3. low'].astype(float)
df['2. high'] = df['2. high'].astype(float)
# print(df['3. low'])
# print(df['2. high'])

# Calculates the average of the 'low' and 'high' values
df['mean_low_high'] = (df['3. low'] + df['2. high']) / 2
# print(df['mean_low_high'])

# Step 1: Calculate the central average
central_average = np.median(df['mean_low_high'])

# Step 2: Calculate deviations from central average
deviations = [abs(x - central_average) for x in df['mean_low_high']]

# Step 4: Calculate MDCA
mdca = np.median(deviations)

# Print the result
# print("MDCA:", mdca)

# Return the result as a string
result_string = f"The MDCA for the given dataset is {mdca:.2f}%."
result_string

'The MDCA for the given dataset is 1.20%.'

In [147]:
# Define the generate_api_invest function
def generate_api_invest(user):
    message = f"The MDCA from {symbol} that you provided is {mdca:.2f}%. Good look at your calls!"
    return message

# Generate investment messages and update news list for each user
for user in users:
    news = generate_api_invest(user)
    print(news)
    # Append the generated message to the user's news list
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

# Print the updated user list
print(json.dumps(users, indent=2))

The MDCA from ibm that you provided is 1.20%. Good look at your calls!
The MDCA from ibm that you provided is 1.20%. Good look at your calls!
[
  {
    "id": 1,
    "name": "Devweekerson",
    "account": {
      "id": 1,
      "number": "01.097954-4",
      "agency": "2030",
      "balance": 624.12,
      "limit": 1000.0
    },
    "card": {
      "id": 1,
      "number": "xxxx xxxx xxxx 1111",
      "limit": 2000.0
    },
    "features": [
      {
        "id": 2,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
        "description": "Pagar"
      },
      {
        "id": 3,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
        "description": "Transferir"
      },
      {
        "id": 4,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
        "description": "Conta Corrente"
      },
      {
        "id": 5,
        "ico

In [154]:
import csv

# Filepath of the CSV file
csv_filepath = "mdca.csv"

# Read existing data from the CSV file
existing_data = []
with open(csv_filepath, 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        existing_data.append(row)

# Append the new data to the existing data
new_row = {mdca, symbol}
existing_data.append(new_row)

# Write the combined data back to the CSV file
with open(csv_filepath, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(existing_data)

print(new_row)

{1.195999999999998, 'ibm'}


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [108]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Devweekerson updated? False!
User Pip updated? True!
